# Build a JSON parser

All rights reserved, 2024 by **Youn-Sik Hong**. 수업 목적으로만 활용 가능.

## 1. Read a JSON data

###  JSON은 python의 dictionary 구조(key-value 쌍)와 비슷합니다.
- JSON은 XML보다 구조가 간단해 데이터를 저장하거나 전송하는 데 유리합니다.

In [1]:
import json

JSON 파일을 직접 읽어올 수 있지만, 여기서는 문자열로 JSON 데이터를 정의했습니다.

In [2]:
json_data = '{"name": "John", "age": 30, "city": "New York"}'

### 1.1 loads(): JSON $\rightarrow$ python dictionary 

In [3]:
parsed_data = json.loads(json_data)
print(parsed_data)

{'name': 'John', 'age': 30, 'city': 'New York'}


- JSON 데이터를 python의 dictonary 구조로 저장했기 때문에 
    - key를 참조해 value를 가져올 수 있습니다.

In [4]:
name = parsed_data['name']
print(f"Name: {name}")

Name: John


### practice 01 : 나머지 key에 대해서도 value를 출력해 보세요.

In [6]:
# 여기에 코드를 작성하세요.
age = parsed_data['age']
print(f"Age: {age}")

city = parsed_data['city']
print(f"City: {city}")

Age: 30
City: New York


### 1.2 dumps() used to serialize a Python object
- 잘 사용하지 않음.

In [7]:
json_data2 = {
    "name": "John", 
    "age": 30, 
    "city": "New York"
}

In [8]:
#dumped_data = json.dumps(json_data2)
dumped_data = json.dumps(json_data2, 
                         indent=4, 
                         sort_keys=True)
print(dumped_data[0:20])

{
    "age": 30,
   


## 2. Build a JSON parser
- JSON의 BNF(ch2(aux).pdf)를 참고하면서 코드를 분석하세요.
    - parse()는 식별 기호에 따라 JSON 구성 요소를 추출하는 각 메소드를 호출합니다.
        - parse()는 parse_object() 및 parse_array()에서도 호출됩니다.
    - parse_boolean()은 python의 literal인 True, False 또는 None을 반환합니다.
    - parse_number()는 float 타입으로 변환합니다.
---    
- 시작 기호 '{', '\[', '"'는 끝 기호'\}', '\]', '"'를 찾아야 합니다.
- 코드를 이해하기 쉽도록 멤버 메소드마다 print()문을 추가했습니다.

In [ ]:
class JSONParser:
    def __init__(self, json_str):
        self.json_str = json_str
        self.index = 0

    def parse(self):
        if self.index >= len(self.json_str):
            raise ValueError("Unexpected end of JSON string")
        ch = self.json_str[self.index]

        if ch == '{':
            return self.parse_object()
        elif ch == '[':
            return self.parse_array()
        elif ch == '"':
            return self.parse_string()
        elif ch.isdigit() or ch == '-':
            return self.parse_number()
        elif ch.isalpha():
            return self.parse_boolean()
        elif ch.isspace() or ch in [',', ':']:
            self.index += 1
            return self.parse()
        else:
            raise ValueError(f"Unexpected character: {ch}")

    def parse_object(self):
        result = {}
        self.index += 1  # Skip curly brace '{'

        while self.json_str[self.index] != '}':
            key = self.parse_string()
            self.index += 1  # Skip colon ':'
            value = self.parse()
            result[key] = value

            if self.json_str[self.index] == ',':
                self.index += 1  # Skip comma ','
        
        self.index += 1  # Skip curly brace '}'
        print(f"parse_object >>> {result}")        
        return result

    def parse_array(self):
        result = []
        self.index += 1  # Skip square bracket '['

        while self.json_str[self.index] != ']':
            value = self.parse()
            result.append(value)
            print(f"parse_array >>> {result}")

            if self.json_str[self.index] == ',':
                self.index += 1  # Skip comma ','
        
        self.index += 1  # Skip square bracket ']'
        return result

    def parse_string(self):
            self.index += 1  # Skip double quote '"'
        start = self.index
        while self.json_str[self.index] != '"':
            self.index += 1
        end = self.index
        self.index += 1  # Skip double quote '"'
        return self.json_str[start:end]

    def parse_number(self):
         start = self.index
        while self.index < len(self.json_str) and (self.json_str[self.index].isdigit() or self.json_str[self.index] in ['.', 'e', 'E', '+', '-']):
        self.index += 1
        end = self.index
        return float(self.json_str[start:end])

    def parse_boolean(self):
        start = self.index
        while self.index < len(self.json_str) and self.json_str[self.index].isalpha():
            self.index += 1
        end = self.index            
        word = self.json_str[start:end]
        print(f"parse_boolean >>> {word}")
        
        if word == 'true':
            return True
        elif word == 'false':
            return False
        elif word == 'null':
            return None
        else:
            raise ValueError(f"Invalid keyword: {word}")

In [10]:
json_sample = '{"name": "John", "age": 30, "city": "New York", "is_student": false, "grades": [90, 85, 92]}'
json_data = JSONParser(json_sample)
parsed_data = json_data.parse()
print(parsed_data)

parse_string >>> n
parse_string >>> name
parse_string >>> J
parse_string >>> John
parse_string >>> a
parse_string >>> age
parse_number >>> 30
parse_string >>> c
parse_string >>> city
parse_string >>> N
parse_string >>> New York
parse_string >>> i
parse_string >>> is_student
parse_boolean >>> false
parse_string >>> g
parse_string >>> grades
parse_number >>> 90
parse_array >>> [90.0]
parse_number >>> 85
parse_array >>> [90.0, 85.0]
parse_number >>> 92
parse_array >>> [90.0, 85.0, 92.0]
parse_object >>> {'name': 'John', 'age': 30.0, 'city': 'New York', 'is_student': False, 'grades': [90.0, 85.0, 92.0]}
{'name': 'John', 'age': 30.0, 'city': 'New York', 'is_student': False, 'grades': [90.0, 85.0, 92.0]}


### practice 02 : JSON data를 직접 만들어 parser를 테스해 보세요.
- 위에서 제시한 예제 수준으로 만들어 보기 바랍니다.

In [15]:
# 여기에 JSON data를 추가하세요.
json_sample = '{"name": "Jiwon", "age": 22, "city": "Daejeon", "is_child": false, "day": [2022, 03, 19]}'
json_data = JSONParser(json_sample)
parsed_data = json_data.parse()
print(parsed_data)

parse_string >>> n
parse_string >>> name
parse_string >>> J
parse_string >>> Jiwon
parse_string >>> a
parse_string >>> age
parse_number >>> 22
parse_string >>> c
parse_string >>> city
parse_string >>> D
parse_string >>> Daejeon
parse_string >>> i
parse_string >>> is_child
parse_boolean >>> false
parse_string >>> d
parse_string >>> day
parse_number >>> 2022
parse_array >>> [2022.0]
parse_number >>> 03
parse_array >>> [2022.0, 3.0]
parse_number >>> 19
parse_array >>> [2022.0, 3.0, 19.0]
parse_object >>> {'name': 'Jiwon', 'age': 22.0, 'city': 'Daejeon', 'is_child': False, 'day': [2022.0, 3.0, 19.0]}
{'name': 'Jiwon', 'age': 22.0, 'city': 'Daejeon', 'is_child': False, 'day': [2022.0, 3.0, 19.0]}


### practice 03 : 아래 JSON 데이터가 컴파일되도록 위 코드를 수정하세요.

In [24]:
json_sample2 = '''
{"name": "John",
 "age": 30,
 "city": "New York",
 "is_student": false,
 "grades": [90, 85, 92]}
'''

In [25]:
json_data2 = JSONParser(json_sample2)
parsed_data2 = json_data2.parse()
print(parsed_data2)

parse_string >>> n
parse_string >>> name
parse_string >>> J
parse_string >>> John
parse_string >>> a
parse_string >>> age
parse_number >>> 30
parse_string >>> c
parse_string >>> city
parse_string >>> N
parse_string >>> New York
parse_string >>> i
parse_string >>> is_student
parse_boolean >>> false
parse_string >>> g
parse_string >>> grades
parse_number >>> 90
parse_array >>> [90.0]
parse_number >>> 85
parse_array >>> [90.0, 85.0]
parse_number >>> 92
parse_array >>> [90.0, 85.0, 92.0]
parse_object >>> {'name': 'John', 'age': 30.0, 'city': 'New York', 'is_student': False, 'grades': [90.0, 85.0, 92.0]}
{'name': 'John', 'age': 30.0, 'city': 'New York', 'is_student': False, 'grades': [90.0, 85.0, 92.0]}
